In [77]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from qiskit.aqua.components.optimizers import COBYLA

## VQE Example with Single Qubit Variational From
Example from: https://qiskit.org/textbook/ch-applications/vqe-molecules.html

"We will now use the simple single qubit variational form to solve a problem similar to ground state energy estimation. Specifically, we are given a random probability vector  $\overrightarrow{x}$ and wish to determine a possible parameterization for our single qubit variational form such that it outputs a probability distribution that is close to $\overrightarrow{x}$ (where closeness is defined in terms of the Manhattan distance between the two probability vectors)."

### Define Target Distribution
we are measuring the probability of the 0 state and the 1 state, so each should be approximately 50/50. Therefore our target distribution is a length two vector with roughly 50% for each component because each number is chosen randomly from 0 to 1. 

In [2]:
np.random.seed(999999)
target_distr = np.random.rand(2)
target_distr /= sum(target_distr)
target_distr

array([0.51357006, 0.48642994])

### Function for Creating Variational Form
It's a circuit with 1 qubit, where the first qubit is acted on by a U3 gate with some input set of parameters.

In [14]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    #print(qc)
    qc.measure(qr, cr[0])
    #print(qc)
    return qc

### Specify Objective Function 
We define the objective function, which is the calculation of the quantity of interest, here it's just a cost function is the sum of the absolute value of the difference between the target distribution and the output distribution of the objective function. 

In [46]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
num_shots = 100

In [6]:
def get_probability_distribution(counts):
    output_distr = [v/num_shots for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

In [75]:
def objective_function(params):
    # Obtain a quantum circuit instance from the parameters
    qc = get_var_form(params)
    #print(qc)
    result = execute(qc, backend, shots=num_shots).result()
    #print(dir(result))
    #print(result.get_counts(qc))
    output_distr = get_probability_distribution(result.get_counts(qc))
    print(output_distr)
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost


In [98]:
params = np.zeros(3)
params[0] = 1.46209037
params[1] = 1.564515
params[2] = 0.34435917
#array([1.46209037, 1.564515  , 0.34435917])
test = objective_function(params)
test

[0.47, 0.53]


0.0871401187388392

In [79]:
# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.001)

In [102]:
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

[0.75, 0.25]
[0.31, 0.69]
[0.26, 0.74]
[0.31, 0.69]
[0.12, 0.88]
[0.21, 0.79]
[0.39, 0.61]
[0.56, 0.44]
[0.66, 0.34]
[0.6, 0.4]
[0.63, 0.37]
[0.62, 0.38]
[0.64, 0.36]
[0.55, 0.45]
[0.6, 0.4]
[0.6, 0.4]
[0.51, 0.49]
[0.43, 0.57]
[0.47, 0.53]
[0.56, 0.44]
[0.55, 0.45]
[0.41, 0.59]
[0.55, 0.45]
[0.56, 0.44]
[0.51, 0.49]
[0.45, 0.55]
[0.47, 0.53]
[0.54, 0.46]
[0.5, 0.5]
[0.6, 0.4]
[0.51, 0.49]


In [110]:
ret[0]
ret[1]
ret[2]

array([1.57432471, 0.24493779, 0.54862615])

0.00714011873883913

31

`ret[0]` is a 1D numpy.ndarray containing the optimal parameters

`ret[1]` is a float with the objective function value (cost in this case)

`ret[2]` number of objective function calls made if available

And now that we have our optimized parameters, we can calculate how close the measurement of the variational form with these parameters is to the target distribution. 

In [112]:
qc = get_var_form(ret[0])
print(qc)

        ┌────────────────────────────┐┌─┐
q_0: |0>┤ U3(1.5743,0.24494,0.54863) ├┤M├
        └────────────────────────────┘└╥┘
 c_0: 0 ═══════════════════════════════╩═
                                         


In [114]:
counts = execute(qc, backend, shots=num_shots).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.49, 0.51]
Output Error (Manhattan Distance): 0.00714011873883913
Parameters Found: [1.57432471 0.24493779 0.54862615]
